In [1]:
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger

In [3]:
# Base User class
class User:
    def __init__(self, name, broker, **kwargs):

        # The User class initializes with name, broker, and other attributes that vary depending on the broker.

        self.name = name
        self.broker = broker

        # Set dynamic attributes based on the broker
        self.username = kwargs.get("username", None)
        self.api_key = kwargs.get("api_key", None)
        self.pwd = kwargs.get("pwd", None)
        self.totp_token = kwargs.get("totp_token", None)
        self.api_obj = None  # Placeholder for api obj
        self.auth_token = None  # Placeholder for login access token
        self.refresh_token = None  # Placeholder for login refresh token
        self.feed_token = None  # Placeholder for login feed token

        # Call the login method automatically when the object is created
        print(self.user_login())

    def get_user_info(self):

        # Returns the user information in a formatted string.
        return f"Name: {self.name}, Broker: {self.broker}, Username: {self.username},"

 # Login Functions
    def user_login(self):

        print(f"Logging in {self.username} to {self.broker}...")

        # Handling Multiple Brokers

        if self.broker == "AngelOne":
            return self._login_angel_one()
        elif self.broker == "OtherBroker":
            return self._login_other_broker()
        else:
            return "Unsupported broker. Please provide valid credentials."

    def _login_angel_one(self):

        # Login using Angel One's SmartAPI and generates an access token.

        try:
            token = self.totp_token
            totp = pyotp.TOTP(token).now()
        except Exception as e:
            logger.error("Invalid Token: The provided token is not valid.")
            raise e

        try:
            # SmartAPI login for Angel One
            self.api_obj = SmartConnect(api_key=self.api_key)

            data = self.api_obj.generateSession(self.username, self.pwd, totp)

            self.auth_token = data['data']['jwtToken']          # Store access token
            self.refresh_token = data['data']['refreshToken']   # Store refresh_token
            self.feed_token = self.api_obj.getfeedToken()       # Store Feed token

            return f"Login {self.api_obj.getProfile(self.refresh_token)["message"]}"

        except Exception as e:
            return f"Angel One login failed: {str(e)}"

    def _login_other_broker(self):
        """
        Placeholder login function for other brokers.
        """

 # Logout Functions
    def user_logout(self):

        # Simulates user logout

        print(f"Logging out {self.name} to {self.broker}...")

        if self.broker == "AngelOne":
            return self._logout_angel_one()
        elif self.broker == "OtherBroker":
            return self._logout_other_broker()
        else:
            return "Logout Failed."

    def _logout_angel_one(self):

        # Logout using Angel One's SmartAPI.

        try:
            # SmartAPI logout for Angel One
            res = self.api_obj.terminateSession(self.username)

            self.auth_token = None       # Reset access token
            self.refresh_token = None    # Reset refresh_token
            self.feed_token = None       # Reset Feed token

            return f"{res["data"]}"

        except Exception as e:
            return f"Angel One logout failed: {str(e)}"

    def _logout_other_broker(self):
        """
        Placeholder logout function for other brokers.
        """

 # Order Placing function

    def place_order(self, order_details):

        #  Handling Multiple Brokers

        if self.broker == "AngelOne":
            return self._place_order_angel_one(order_details)
        elif self.broker == "OtherBroker":
            return self._place_order_other_broker(order_details)
        else:
            return "Unsupported broker."

    def _place_order_angel_one(self, order_details):
        # placing order using Angel One's SmartAPI
        try:
            # # Method 1: Place an order and return the order ID
            # res = self.api_obj.placeOrder(order_details)

            # Method 2: Place an order and return the full response
            res = self.api_obj.placeOrderFullResponse(order_details)

            return f"PlaceOrder : {res}"

        except Exception as e:
            return f"Order placement failed: {e}"

    def _place_order_other_broker(self, order_details):
        """
        Placeholder order function for other brokers.
        """

# Parent User class with broker-specific attributes and children management
class ParentUser(User):
    def __init__(self, name, broker, **kwargs):

        # ParentUser class inherits from User and can manage child users.

        super().__init__(name, broker, **kwargs)
        self.children = []

    def add_child(self, child):

        # Adds a ChildUser to the ParentUser's list of children.

        self.children.append(child)

    def list_children(self):

        # Returns a list of the usernames of all children under the ParentUser.

        return [child.name for child in self.children]

    def place_order_for_child(self, order_details):
         # Places an order for all children.
        for child in self.children:
            child_order_result = child.place_order(order_details)
            print(f"Child Order: {child_order_result}")

    def place_order_for_all(self, order_details):

        # Places an order for both the parent and all children.

        parent_order_result = self.place_order(order_details)
        print(parent_order_result)
        self.place_order_for_child(order_details)

# Child User class (must be linked to a ParentUser)
class ChildUser(User):
    def __init__(self, name, broker, parent, **kwargs):

        # ChildUser class inherits from User and is linked to a ParentUser.

        super().__init__(name, broker, **kwargs)
        self.parent = parent
        parent.add_child(self)  # Automatically add the child to the parent's list

    def get_parent_info(self):

        # Returns the parent user's information.

        return f"Parent Name: {self.parent.name}, Parent Broker: {self.parent.broker}"


In [4]:
# Example Usage
# Creating a ParentUser with custom attributes based on the broker
parent = ParentUser(name="Yash",
                    broker="AngelOne",
                    api_key = 'bE7ipU6q',
                    username = 'Y58983127',
                    pwd = '1416',
                    totp_token = "NQST6OLTU6MP3PKRH2BLXUTGDI")

Logging in Y58983127 to AngelOne...


[I 241019 03:03:18 smartConnect:121] in pool


Login SUCCESS


In [5]:
# Creating ChildUser instances and linking them to the parent
child1 = ChildUser( name="Ravi",
                    broker="AngelOne",
                    parent=parent,
                    api_key = 'eskMiA2H',
                    username = 'R947822',
                    pwd = '1234',
                    totp_token = "E4FCOCHUNVRDG7ANFXK6D3OQVA")

Logging in R947822 to AngelOne...


[I 241019 03:03:27 smartConnect:121] in pool


Login SUCCESS


In [6]:
# Print user info
print(child1.get_parent_info())       # Parent user information
print(parent.list_children())         # Children user information

Parent Name: Yash, Parent Broker: AngelOne
['Ravi']


In [ ]:
# Placing an order for both parent and children
parent.place_order_for_all(order_details={"symbol": "AAPL", "qty": 10, "action": "buy"})

In [7]:
order_details = {
                "variety": "NORMAL",
                "tradingsymbol": "SBIN-EQ",
                "symboltoken": "3045",
                "transactiontype": "BUY",
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0",
                "squareoff": "0",
                "stoploss": "0",
                "quantity": "1"
                }

parent.place_order(order_details=order_details)

"PlaceOrder : {'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'script': 'SBIN-EQ', 'orderid': '241019100000156', 'uniqueorderid': 'faffcbe7-43ed-443a-a9ff-ad4c47625783'}}"

In [ ]:
parent.api_obj.cancelOrder('241019100000156','NORMAL')

[E 241019 03:12:08 smartConnect:243] Error occurred while making a POST request to https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/cancelOrder. Error: OrderId not found. URL: https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/cancelOrder, Headers: {'Content-type': 'application/json', 'X-ClientLocalIP': '127.0.0.1', 'X-ClientPublicIP': '106.193.147.98', 'X-MACAddress': '1c:4d:70:be:5e:50', 'Accept': 'application/json', 'X-PrivateKey': 'bE7ipU6q', 'X-UserType': 'USER', 'X-SourceID': 'WEB'}, Request: {'variety': 'NORMAL', 'orderid': 'faffcbe7-43ed-443a-a9ff-ad4c47625783'}, Response: {'message': 'OrderId not found', 'errorcode': 'AB3001', 'status': False, 'data': None}


{'message': 'OrderId not found',
 'errorcode': 'AB3001',
 'status': False,
 'data': None}

In [10]:
parent.api_obj.individual_order_details('faffcbe7-43ed-443a-a9ff-ad4c47625783')

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': {'variety': 'NORMAL',
  'ordertype': 'MARKET',
  'producttype': 'INTRADAY',
  'duration': 'DAY',
  'price': 0.0,
  'triggerprice': 0.0,
  'quantity': '1',
  'disclosedquantity': '0',
  'squareoff': 0.0,
  'stoploss': 0.0,
  'trailingstoploss': 0.0,
  'tradingsymbol': 'SBIN-EQ',
  'transactiontype': 'BUY',
  'exchange': 'NSE',
  'symboltoken': '3045',
  'instrumenttype': '',
  'strikeprice': -1.0,
  'optiontype': '',
  'expirydate': '',
  'lotsize': '1',
  'cancelsize': '0',
  'averageprice': 0.0,
  'filledshares': '0',
  'unfilledshares': '1',
  'orderid': '',
  'text': '',
  'status': 'open',
  'orderstatus': 'open',
  'updatetime': '',
  'exchtime': '',
  'exchorderupdatetime': '',
  'fillid': '',
  'filltime': '',
  'parentorderid': '',
  'ordertag': '',
  'uniqueorderid': 'faffcbe7-43ed-443a-a9ff-ad4c47625783'}}

In [12]:
parent.user_logout()


Logging out Yash to AngelOne...


'Logout Successfully'

In [13]:
child1.user_logout()

Logging out Ravi to AngelOne...


'Logout Successfully'

In [33]:
from SmartApi.smartWebSocketOrderUpdate import SmartWebSocketOrderUpdate

sws = SmartWebSocketOrderUpdate(parent.auth_token,parent.api_key, parent.username, parent.feed_token)